In [1]:
import pandas as pd
import arff as liacarff

# 加载 ARFF 文件
with open('PROMISE_exp.arff', 'r') as f:
    data_dict = liacarff.load(f)

# 提取数据
data = data_dict['data']

# 转换为 DataFrame
df = pd.DataFrame(data, columns=[attr[0] for attr in data_dict['attributes']])
df['ProjectID'] = df['ProjectID'].astype(int)  # 将 ProjectID 列转换为整数类型

# 显示 DataFrame
print(df)


     ProjectID                                    RequirementText _class_
0            1  The system shall refresh the display every 60 ...      PE
1            1  The application shall match the color of the s...      LF
2            1  If projected  the data must be readable.  On a...      US
3            1  The product shall be available during normal b...       A
4            1  If projected  the data must be understandable....      US
..         ...                                                ...     ...
965         48  Registered User must be able to maintain his/h...       F
966         48  The entire website must be user-friendly and e...      US
967         48  The system shall support up to 10000 simultane...      PE
968         48  The website must provide highest degree of sec...      SE
969         49  The software application should be easily tran...      PO

[970 rows x 3 columns]


In [2]:
# DATA PREPROCESSING
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import string

# 下载NLTK的停用词和词性标注器数据
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# lower
df['RequirementText'] = df['RequirementText'].str.lower()

# Remove punctuation, leading and trailing spaces
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(f"[{re.escape(string.punctuation)}]", "", x.strip()))

# 分词
df['RequirementText'] = df['RequirementText'].apply(word_tokenize)

# stopwords
stop_words = set(stopwords.words('english'))
df['RequirementText'] = df['RequirementText'].apply(lambda x: [word for word in x if word not in stop_words])

# stemming
# stemmer = PorterStemmer()
# df['RequirementText'] = df['RequirementText'].apply(lambda x: [stemmer.stem(word) for word in x])

# lemmatization
lemmatizer = WordNetLemmatizer()
df['RequirementText'] = df['RequirementText'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# 合并词语为字符串
df['RequirementText'] = df['RequirementText'].apply(lambda x: ' '.join(x))

# Replace multiple spaces with a single space
df['RequirementText'] = df['RequirementText'].apply(lambda x: re.sub(' +', ' ', x))

# 移除整个列中每个字符串首尾的全部空格
df['RequirementText'] = df['RequirementText'].str.strip()

# 查看处理后的数据集
print(df.head())

[nltk_data] Downloading package stopwords to /home/li/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/li/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/li/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


   ProjectID                                    RequirementText _class_
0          1       system shall refresh display every 60 second      PE
1          1  application shall match color schema set forth...      LF
2          1  projected data must readable 10x10 projection ...      US
3          1  product shall available normal business hour l...       A
4          1  projected data must understandable 10x10 proje...      US


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [4]:
class_distribution = df['_class_'].value_counts()
print(class_distribution)

_class_
F     444
SE    125
US     85
O      77
PE     67
LF     49
A      31
MN     24
SC     22
FT     18
L      15
PO     13
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split
y = df['_class_']
X = df['RequirementText']

# 假设 X 是特征，y 是目标变量（类别标签）
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y)

print(y_test)
class_distribution = y_test.value_counts()
print(class_distribution)

19      F
572    MN
30     LF
751     F
884     F
       ..
246     F
208    LF
822     F
631     F
515     F
Name: _class_, Length: 97, dtype: object
_class_
F     44
SE    13
US     9
O      8
PE     7
LF     5
A      3
MN     2
SC     2
FT     2
PO     1
L      1
Name: count, dtype: int64


In [7]:
# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [8]:
model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

MultinomialNB()

In [9]:
y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

# 评估模型性能
print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

Accuracy (TF-IDF): 0.5979381443298969
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       0.00      0.00      0.00         3
           F       0.61      0.98      0.75        44
          FT       0.00      0.00      0.00         2
           L       0.00      0.00      0.00         1
          LF       1.00      0.20      0.33         5
          MN       0.00      0.00      0.00         2
           O       0.60      0.38      0.46         8
          PE       1.00      0.14      0.25         7
          PO       0.00      0.00      0.00         1
          SC       0.00      0.00      0.00         2
          SE       0.64      0.54      0.58        13
          US       0.33      0.33      0.33         9

    accuracy                           0.60        97
   macro avg       0.35      0.21      0.23        97
weighted avg       0.57      0.60      0.52        97



/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# F binary

df['binary_label'] = df['_class_'].apply(lambda x: 1 if x == 'F' else 0)
X_train, X_test, y_train, y_test = train_test_split(df['RequirementText'], df['binary_label'], test_size=0.2, random_state=42)

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

Accuracy (TF-IDF): 0.8144329896907216
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           0       0.79      0.87      0.83       102
           1       0.84      0.75      0.79        92

    accuracy                           0.81       194
   macro avg       0.82      0.81      0.81       194
weighted avg       0.82      0.81      0.81       194



In [11]:
filtered_df = df[df['_class_'] != 'F']
X_train, X_test, y_train, y_test = train_test_split(filtered_df['RequirementText'], filtered_df['_class_'], test_size=0.2, random_state=42, stratify=filtered_df['_class_'])

# 使用TfidfVectorizer进行转换
ngram_range = (1,2)
min_df = 0.01 # ignore terms that appear in less than 1% of the documents
max_df = 0.8 # ignore terms that appear in more than 80% of the documents

tfidf_vectorizer = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        norm='l2',
                        sublinear_tf=True)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

model_tfidf = MultinomialNB()
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)

print("Accuracy (TF-IDF):", accuracy_score(y_test, y_pred_tfidf))
print("Classification Report (TF-IDF):\n", classification_report(y_test, y_pred_tfidf))

Accuracy (TF-IDF): 0.5471698113207547
Classification Report (TF-IDF):
               precision    recall  f1-score   support

           A       1.00      0.17      0.29         6
          FT       0.00      0.00      0.00         4
           L       0.00      0.00      0.00         3
          LF       0.33      0.10      0.15        10
          MN       0.00      0.00      0.00         5
           O       0.50      0.75      0.60        16
          PE       0.60      0.69      0.64        13
          PO       0.00      0.00      0.00         3
          SC       0.00      0.00      0.00         4
          SE       0.54      0.84      0.66        25
          US       0.58      0.82      0.68        17

    accuracy                           0.55       106
   macro avg       0.32      0.31      0.27       106
weighted avg       0.46      0.55      0.46       106



/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/li/dsse/indi/indi/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
